#  모델 학습 (SoftMax, Decision Tree, Random Forest )


## 1. 모듈 호출

In [87]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
#from pycaret.classification import *
from sklearn.decomposition import PCA
import torch
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import os

## 2. 데이터 호출 및 정제


In [151]:
data = pd.read_csv("labels_included.csv")
svd_data= pd.read_csv("SVD_Webtoon_Data.csv")
label_data =pd.read_csv("웹툰 라벨링 데이터_2.csv")
data

,rank_popularity,webtoon_name,rating,link,genre,interest,episode_count,originality,authors,work_count,...,행복,화남/분노,환영/호의,흐뭇함(귀여움/예쁨),힘듦/지침,day,rank_day,scaled_interest,scaled_rank,cluster
0,89.0,노블레스,9.62,https://comic.naver.com/webtoon/list?titleId=2...,판타지,1371554,545,0,"['손제호', '이광수']",3.500000,...,0.0,0.000000,0.000000,0.000000,0.0,NaN,NaN,9198.671890,961.736887,7.0
1,48.0,갓 오브 하이스쿨,9.64,https://comic.naver.com/webtoon/list?titleId=3...,판타지,1371113,573,0,['박용제'],2.000000,...,0.0,0.000000,0.000000,0.013333,0.0,NaN,NaN,9195.681202,979.363715,7.0
2,28.0,유미의 세포들,9.93,https://comic.naver.com/webtoon/list?titleId=6...,로맨스,1285627,512,0,['이동건'],11.000000,...,0.0,0.066667,0.013333,0.026667,0.0,NaN,NaN,8615.948845,987.962167,7.0
3,270.0,여신강림,9.34,https://comic.naver.com/webtoon/list?titleId=7...,로맨스,1216760,261,0,['야옹이'],4.000000,...,0.0,0.000000,0.000000,0.053333,0.0,NaN,NaN,8148.919995,883.920894,7.0
4,429.0,싸움독학,9.67,https://comic.naver.com/webtoon/list?titleId=7...,액션,1115468,223,0,"['박만사', '김정현']",13.500000,...,0.0,0.013333,0.000000,0.013333,0.0,NaN,NaN,7461.997576,815.563199,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,NaN,노래 못 하는 남자,9.92,https://comic.naver.com/webtoon/list?titleId=8...,로맨스,22545,57,0,"['재은', '이해금']",1.000000,...,0.0,0.032258,0.000000,0.064516,0.0,sun,92.0,50.224573,30.303030,0.0
1203,NaN,마섹남 - 마술하는 섹시한 남자,9.89,https://comic.naver.com/webtoon/list?titleId=7...,로맨스,54426,90,1,"['랑또', '육시몬', '티르스']",4.000000,...,0.0,0.014493,0.014493,0.028986,0.0,sun,93.0,266.428949,23.088023,0.0
1204,NaN,주인공의 주식을 팝니다,9.84,https://comic.naver.com/webtoon/list?titleId=8...,로맨스,57199,96,1,"['목감기', '가얌', '50']",1.333333,...,0.0,0.017857,0.000000,0.071429,0.0,sun,94.0,285.234342,12.987013,0.0
1205,NaN,프린키피아,9.84,https://comic.naver.com/webtoon/list?titleId=8...,판타지,23203,66,0,"['버튼', '한경찰']",3.500000,...,0.0,0.000000,0.000000,0.045455,0.0,sun,95.0,54.686870,141.444540,0.0


In [152]:
data.drop(['interest','cluster','day','rank_day','rank_popularity','link','authors','scaled_rank','scaled_interest','episode_count','work_count'],axis=1,inplace=True)
data=data.drop_duplicates()
data

,webtoon_name,rating,genre,originality,previous_work,likes,rating_participants,comments_count,감동/감탄,경악,...,즐거움/신남,증오/혐오,지긋지긋,짜증,한심함,행복,화남/분노,환영/호의,흐뭇함(귀여움/예쁨),힘듦/지침
0,노블레스,9.62,판타지,0,0.0,91983,263859,82093,0.133333,0.000000,...,0.133333,0.000000,0.0,0.026667,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,갓 오브 하이스쿨,9.64,판타지,0,1.0,134823,261048,56406,0.040000,0.000000,...,0.106667,0.000000,0.0,0.000000,0.013333,0.0,0.000000,0.000000,0.013333,0.0
2,유미의 세포들,9.93,로맨스,0,1.0,355753,211459,32296,0.120000,0.013333,...,0.160000,0.000000,0.0,0.106667,0.026667,0.0,0.066667,0.013333,0.026667,0.0
3,여신강림,9.34,로맨스,0,0.0,373202,356731,41199,0.080000,0.000000,...,0.106667,0.000000,0.0,0.066667,0.040000,0.0,0.000000,0.000000,0.053333,0.0
4,싸움독학,9.67,액션,0,1.0,265394,197599,17442,0.240000,0.000000,...,0.240000,0.013333,0.0,0.000000,0.000000,0.0,0.013333,0.000000,0.013333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,밤필드의 아이들 by DARK MOON,9.84,판타지,0,1.0,11676,7777,2513,0.112676,0.000000,...,0.112676,0.000000,0.0,0.014085,0.000000,0.0,0.000000,0.000000,0.084507,0.0
1202,노래 못 하는 남자,9.92,로맨스,0,0.0,4658,3955,282,0.129032,0.000000,...,0.000000,0.000000,0.0,0.032258,0.000000,0.0,0.032258,0.000000,0.064516,0.0
1203,마섹남 - 마술하는 섹시한 남자,9.89,로맨스,1,1.0,21651,17845,1257,0.130435,0.000000,...,0.188406,0.000000,0.0,0.014493,0.000000,0.0,0.014493,0.014493,0.028986,0.0
1204,주인공의 주식을 팝니다,9.84,로맨스,1,1.0,27970,21482,1275,0.196429,0.000000,...,0.160714,0.000000,0.0,0.000000,0.000000,0.0,0.017857,0.000000,0.071429,0.0


In [153]:
label_data.rename(columns={"웹툰 이름":'webtoon_name'},inplace=True)
label_data=label_data[['webtoon_name','정답 라벨링']]
label_data

,webtoon_name,정답 라벨링
0,노블레스,5
1,갓 오브 하이스쿨,5
2,유미의 세포들,5
3,여신강림,5
4,싸움독학,5
...,...,...
2246,밤필드의 아이들 by DARK MOON,1
2247,노래 못 하는 남자,1
2248,마섹남 - 마술하는 섹시한 남자,1
2249,주인공의 주식을 팝니다,1


In [154]:
svd_data.rename(columns={"웹툰 이름":'webtoon_name'},inplace=True)
svd_data.drop(['Unnamed: 0','장르', '웹툰 관심수', '원작 유무', '전작 유무'],axis=1,inplace=True)
svd_data

,webtoon_name,Info0,Info1,Info2,Info3,Info4,Info5,Info6,Info7,Info8,...,Tag0,Tag1,Tag2,Tag3,Tag4,Tag5,Tag6,Tag7,Tag8,Tag9
0,참교육,10.237783,0.420443,0.246244,-0.650733,-0.093337,-0.411505,0.679820,-0.341738,0.284442,...,1.171455,-0.047548,-0.053223,0.017942,0.077060,-0.445304,-0.186483,-0.141557,0.559626,0.192245
1,환생천마,10.207592,-1.836704,0.892227,-0.604062,0.137026,-0.297536,-1.133568,0.310458,0.369579,...,1.217058,-0.063478,-0.021855,-0.008416,0.180682,-0.574965,-0.282596,-0.444715,-0.069309,0.087533
2,똑 닮은 딸,9.872365,-2.246858,0.113027,0.176220,0.517358,-0.636351,-1.057912,0.028284,0.225315,...,1.113890,-0.042252,-0.055632,0.008371,0.022143,-0.564991,-0.213061,0.288990,0.987908,0.174486
3,신의 탑,8.671783,3.247978,-1.515203,-1.087246,0.789852,-0.841804,-0.302751,-0.007835,-0.067117,...,1.307903,-0.056197,-0.048262,0.012111,0.213633,-0.397434,-0.050867,-0.894449,-0.311774,-0.355934
4,어느날 갑자기 가슴이 커짐,9.505161,-2.034320,-1.945097,1.424878,-0.040076,0.413971,0.261262,0.998126,-0.752153,...,0.154537,0.010050,0.000139,0.020883,-0.060519,0.364899,0.245642,-0.058424,0.086896,0.072552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,블로섬 데이즈,9.648355,-1.472480,-1.919009,0.828522,-0.071013,-0.033091,0.109511,0.298364,-0.354487,...,0.211910,0.312873,0.873128,0.097363,0.000513,0.089857,0.128083,-0.491036,-0.131050,-0.524520
3057,메이드 인 헤븐,9.759952,-2.523996,-0.721804,0.537661,0.818270,-1.073711,-0.546116,0.584503,0.133139,...,0.041903,0.065279,-0.017721,-0.072470,-0.001038,-0.547462,0.829608,0.243734,0.021288,-0.033300
3058,치렐루야,9.598195,-1.511001,3.314267,-0.765617,2.110518,0.897339,0.190554,-0.014322,-0.171462,...,0.156268,0.158128,-0.041423,-0.118071,-0.124762,0.339971,0.193529,-0.175314,-0.291863,0.461805
3059,아마데우스(Amadeus),10.008545,0.097921,-0.857601,0.357149,-0.429522,1.810359,-0.147561,-0.134760,-1.378360,...,0.016449,0.487918,-0.276848,0.800942,0.063432,-0.019232,-0.023652,0.044583,-0.006200,-0.006689


In [155]:
new_data=pd.merge(data, svd_data, on='webtoon_name', how='inner')
new_data_2=pd.merge(new_data,label_data,on='webtoon_name', how='inner')
data=new_data_2

In [156]:
data.drop_duplicates('webtoon_name',inplace=True)
data.reset_index(drop=True,inplace=True)
data

,webtoon_name,rating,genre,originality,previous_work,likes,rating_participants,comments_count,감동/감탄,경악,...,Tag1,Tag2,Tag3,Tag4,Tag5,Tag6,Tag7,Tag8,Tag9,정답 라벨링
0,노블레스,9.62,판타지,0,0.0,91983,263859,82093,0.133333,0.000000,...,0.123178,-0.013558,-0.195868,1.011551,0.118073,-0.004344,0.124505,0.051349,-0.023565,5
1,갓 오브 하이스쿨,9.64,판타지,0,1.0,134823,261048,56406,0.040000,0.000000,...,0.142793,-0.016521,-0.172186,1.100053,0.080604,0.016912,0.108956,0.073123,-0.021607,5
2,유미의 세포들,9.93,로맨스,0,1.0,355753,211459,32296,0.120000,0.013333,...,0.352287,0.872996,0.156310,-0.036568,-0.036958,0.037249,0.026779,0.115661,-0.285022,5
3,여신강림,9.34,로맨스,0,0.0,373202,356731,41199,0.080000,0.000000,...,0.361980,0.935466,0.157913,-0.067107,-0.012034,0.031867,0.042018,0.178152,-0.321376,5
4,싸움독학,9.67,액션,0,1.0,265394,197599,17442,0.240000,0.000000,...,0.044292,-0.002515,0.015135,0.059810,-0.026418,0.074735,-0.205353,0.051129,0.014334,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,밤필드의 아이들 by DARK MOON,9.84,판타지,0,1.0,11676,7777,2513,0.112676,0.000000,...,0.014969,0.030298,0.025214,0.174174,-0.014689,0.156510,-0.695191,-0.030187,-0.178644,1
1183,노래 못 하는 남자,9.92,로맨스,0,0.0,4658,3955,282,0.129032,0.000000,...,-0.048709,-0.014339,-0.009895,-0.072183,0.048326,-0.048009,0.265242,0.086099,-0.003783,1
1184,마섹남 - 마술하는 섹시한 남자,9.89,로맨스,1,1.0,21651,17845,1257,0.130435,0.000000,...,-0.070801,-0.006190,-0.018192,-0.107150,-0.084214,-0.215767,0.551927,0.102878,0.077217,1
1185,주인공의 주식을 팝니다,9.84,로맨스,1,1.0,27970,21482,1275,0.196429,0.000000,...,-0.090322,-0.053630,-0.030389,-0.052154,-0.457863,-0.554196,0.787461,-0.793389,-0.241040,1


In [157]:
# 댓글 감정 분석 결과 PCA


# 차원 축소 대상 열 리스트
emotion_columns = [
    '감동/감탄', '경악', '고마움', '공포/무서움', '귀찮음', '기대감', '기쁨', '깨달음', '놀람', 
    '당황/난처', '부담/안_내킴', '불쌍함/연민', '불안/걱정', '불평/불만', '비장함', '슬픔', '신기함/관심',
    '아껴주는', '안심/신뢰', '안타까움/실망', '어이없음', '없음', '우쭐댐/무시함', '의심/불신', '재미없음',
    '절망', '존경', '즐거움/신남', '증오/혐오', '지긋지긋', '짜증', '한심함', '행복', '화남/분노',
    '환영/호의', '흐뭇함(귀여움/예쁨)', '힘듦/지침'
]

# 대상 열을 스케일링
scaler = MinMaxScaler()
scaled_emotions = scaler.fit_transform(data[emotion_columns])

# PCA 모델 초기화 (설명된 분산 비율 기준)
pca = PCA(n_components=0.95)  # 데이터의 95% 설명력을 유지하도록 설정
reduced_emotions = pca.fit_transform(scaled_emotions)

# 주성분 DataFrame 생성
pca_columns = [f'PCA_{i+1}' for i in range(reduced_emotions.shape[1])]
pca_df = pd.DataFrame(reduced_emotions, columns=pca_columns)

# 기존 데이터프레임과 합치기
data_cleaned_reduced = pd.concat([data.drop(columns=emotion_columns), pca_df], axis=1)

# 결과 확인
print(f"PCA 적용 후 차원 축소: {len(emotion_columns)} -> {reduced_emotions.shape[1]} 차원")
data_cleaned_reduced

PCA 적용 후 차원 축소: 37 -> 29 차원


,webtoon_name,rating,genre,originality,previous_work,likes,rating_participants,comments_count,Info0,Info1,...,PCA_20,PCA_21,PCA_22,PCA_23,PCA_24,PCA_25,PCA_26,PCA_27,PCA_28,PCA_29
0,노블레스,9.62,판타지,0,0.0,91983,263859,82093,10.606161,0.769629,...,-0.038322,0.102593,-0.085764,-0.024303,-0.008405,-0.074357,-0.025306,0.021749,-0.040370,0.009154
1,갓 오브 하이스쿨,9.64,판타지,0,1.0,134823,261048,56406,9.989232,0.971496,...,-0.152914,-0.098255,0.040238,-0.028841,0.042227,0.044980,0.019082,0.094946,0.116015,0.288684
2,유미의 세포들,9.93,로맨스,0,1.0,355753,211459,32296,8.541318,3.894672,...,0.060414,-0.096522,-0.052003,0.049616,0.198412,-0.014719,-0.106958,0.129598,-0.031860,-0.016623
3,여신강림,9.34,로맨스,0,0.0,373202,356731,41199,9.301664,2.420933,...,0.041666,0.082023,0.047631,0.032247,-0.031933,-0.063547,-0.024280,-0.000441,-0.024886,0.005764
4,싸움독학,9.67,액션,0,1.0,265394,197599,17442,10.306073,1.749830,...,-0.052893,0.009277,0.008706,-0.151124,-0.028464,-0.055327,-0.014946,-0.042157,-0.012861,-0.011364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,밤필드의 아이들 by DARK MOON,9.84,판타지,0,1.0,11676,7777,2513,10.569433,-0.395996,...,-0.163799,-0.041973,0.048846,0.057402,0.145379,-0.084544,-0.043655,-0.008827,0.003318,-0.005393
1183,노래 못 하는 남자,9.92,로맨스,0,0.0,4658,3955,282,10.360507,-1.450214,...,-0.008234,-0.038145,-0.184873,-0.001292,0.087442,-0.052468,-0.024067,-0.090273,-0.006691,0.071030
1184,마섹남 - 마술하는 섹시한 남자,9.89,로맨스,1,1.0,21651,17845,1257,9.467278,-2.523441,...,-0.058095,0.055444,-0.023034,0.068674,0.080686,-0.066182,-0.034805,-0.022161,-0.040053,-0.011946
1185,주인공의 주식을 팝니다,9.84,로맨스,1,1.0,27970,21482,1275,9.224359,-1.108212,...,-0.053322,-0.140286,-0.068513,0.060135,0.144034,-0.034755,-0.013175,-0.052715,0.003810,-0.015075


In [ ]:
data_cleaned_reduced.drop()

In [158]:
#웹툰제목을 인덱스로 설정
data=data_cleaned_reduced.set_index("webtoon_name")
data['cluster']=list(map(str,data['정답 라벨링']))

In [160]:
# RandomOverSampler
# 디바이스 설정
device = 1 if torch.cuda.is_available() else -1

# 데이터 준비 (target 열을 지정하세요)
X = data.drop(['정답 라벨링','cluster','rating'],axis=1)
features = pd.get_dummies(X, columns=['genre'], prefix='genre')
labels = data["cluster"]

from imblearn.over_sampling import RandomOverSampler

#  데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
     features, labels, test_size=0.2, random_state=21, stratify=labels
 )

# Random Over Sampling 적용
ros = RandomOverSampler(random_state=21)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

## 3. 그리드서치 CV

In [161]:
best_estimators = {}
best_scores = {}

# 1. Softmax Regression (Logistic Regression)
print("Tuning Logistic Regression (Softmax)...")
logistic_param_grid = {
    'C': [0.1, 1, 10],  # 정규화 강도
    'solver': ['lbfgs', 'newton-cg'],  # 최적화 알고리즘
    'multi_class': ['multinomial'],  # 소프트맥스 사용
    'max_iter': [100, 200]  # 최대 반복 횟수
}
logistic_model = LogisticRegression()
logistic_search = GridSearchCV(logistic_model, logistic_param_grid, cv=5, scoring='f1_macro', verbose=1)
logistic_search.fit(X_train_ros, y_train_ros)
best_estimators['LogisticRegression'] = logistic_search.best_estimator_
best_scores['LogisticRegression'] = logistic_search.best_score_

# 2. Decision Tree
print("Tuning Decision Tree...")
tree_param_grid = {
    'criterion': ['gini', 'entropy'],  # 분할 기준
    'max_depth': [3, 5, None],  # 최대 깊이
    'min_samples_split': [2, 5, 10]  # 분할 최소 샘플 수
}
tree_model = DecisionTreeClassifier(random_state=21)
tree_search = GridSearchCV(tree_model, tree_param_grid, cv=5, scoring='f1_macro', verbose=1)
tree_search.fit(X_train_ros, y_train_ros)
best_estimators['DecisionTree'] = tree_search.best_estimator_
best_scores['DecisionTree'] = tree_search.best_score_

# 3. Random Forest
print("Tuning Random Forest...")
forest_param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [3, 5, None],  # 최대 깊이
    'min_samples_split': [2, 5, 10],  # 분할 최소 샘플 수
    'criterion': ['gini', 'entropy']  # 분할 기준
}
forest_model = RandomForestClassifier(random_state=21)
forest_search = GridSearchCV(forest_model, forest_param_grid, cv=5, scoring='f1_macro', verbose=1)
forest_search.fit(X_train_ros, y_train_ros)
best_estimators['RandomForest'] = forest_search.best_estimator_
best_scores['RandomForest'] = forest_search.best_score_

# 최적 모델과 결과 출력
print("\nBest Models and Scores:")
for model_name, model in best_estimators.items():
    print(f"{model_name}:")
    print(f"  Best Parameters: {model}")
    print(f"  Best CV Score: {best_scores[model_name]:.4f}")

# 최적 모델로 테스트 세트 평가
print("\nTest Set Performance:")
for model_name, model in best_estimators.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred, target_names=target_name))

Tuning Logistic Regression (Softmax)...
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.or

Tuning Decision Tree...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Tuning Random Forest...
Fitting 5 folds for each of 54 candidates, totalling 270 fits

Best Models and Scores:
LogisticRegression:
  Best Parameters: LogisticRegression(C=10, max_iter=200, multi_class='multinomial',
                   solver='newton-cg')
  Best CV Score: 0.7273
DecisionTree:
  Best Parameters: DecisionTreeClassifier(criterion='entropy', random_state=21)
  Best CV Score: 0.8585
RandomForest:
  Best Parameters: RandomForestClassifier(n_estimators=200, random_state=21)
  Best CV Score: 0.8974

Test Set Performance:
LogisticRegression Test Accuracy: 0.5336
              precision    recall  f1-score   support

           4       0.63      0.75      0.69        32
           2       0.44      0.52      0.48        56
           3       0.67      0.46      0.54        92
           5       0.48      0.53      0.51        45
           1       0.38      0.62      0.47        13

    accuracy 

괜찮은 성능이 나왔지만, 테스트 횟수를 늘려보고, 
일부 피쳐 엔지니어링을 진행하자.
그리고 스코어링을 f1_macro로 하여 다중 분류에서 유효한 성능 지표를 기준으로 찾게 변환해본다


In [96]:

data['originality']=list(map(bool,data['originality']))
data['previous_work']=list(map(bool,data['previous_work']))

X=data[list(data.columns)].drop(['interest','cluster','day','rank_day','rank_popularity','link','authors','scaled_rank','scaled_interest','episode_count','work_count'],axis=1)
y=data['cluster']

X = pd.get_dummies(X, columns=['genre'], prefix='genre')

X_train, X_test, y_train, y_test = train_test_split(X, data['cluster'], test_size=0.2, random_state=21)
target_name = list(set(y))
target_name

best_estimators = {}
best_scores = {}

# 1. Softmax Regression (Logistic Regression)
print("Tuning Logistic Regression (Softmax)...")
logistic_param_grid = {
    'C': [0.1, 1, 10],  # 정규화 강도
    'solver': ['lbfgs', 'newton-cg'],  # 최적화 알고리즘
    'multi_class': ['multinomial'],  # 소프트맥스 사용
    'max_iter': [100, 200]  # 최대 반복 횟수
}
logistic_model = LogisticRegression()
logistic_search = GridSearchCV(logistic_model, logistic_param_grid, cv=5, scoring='f1_macro', verbose=1)
logistic_search.fit(X_train, y_train)
best_estimators['LogisticRegression'] = logistic_search.best_estimator_
best_scores['LogisticRegression'] = logistic_search.best_score_

# 2. Decision Tree
print("Tuning Decision Tree...")
tree_param_grid = {
    'criterion': ['gini', 'entropy'],  # 분할 기준
    'max_depth': [3, 5, None],  # 최대 깊이
    'min_samples_split': [2, 5, 10]  # 분할 최소 샘플 수
}
tree_model = DecisionTreeClassifier(random_state=21)
tree_search = GridSearchCV(tree_model, tree_param_grid, cv=5, scoring='f1_macro', verbose=1)
tree_search.fit(X_train, y_train)
best_estimators['DecisionTree'] = tree_search.best_estimator_
best_scores['DecisionTree'] = tree_search.best_score_

# 3. Random Forest
print("Tuning Random Forest...")
forest_param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [3, 5, None],  # 최대 깊이
    'min_samples_split': [2, 5, 10],  # 분할 최소 샘플 수
    'criterion': ['gini', 'entropy']  # 분할 기준
}
forest_model = RandomForestClassifier(random_state=21)
forest_search = GridSearchCV(forest_model, forest_param_grid, cv=5, scoring='f1_macro', verbose=1)
forest_search.fit(X_train, y_train)
best_estimators['RandomForest'] = forest_search.best_estimator_
best_scores['RandomForest'] = forest_search.best_score_

# 최적 모델과 결과 출력
print("\nBest Models and Scores:")
for model_name, model in best_estimators.items():
    print(f"{model_name}:")
    print(f"  Best Parameters: {model}")
    print(f"  Best CV Score: {best_scores[model_name]:.4f}")

# 최적 모델로 테스트 세트 평가
print("\nTest Set Performance:")
for model_name, model in best_estimators.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred, target_names=target_name))

KeyError: "['interest', 'day', 'rank_day', 'rank_popularity', 'link', 'authors', 'scaled_rank', 'scaled_interest', 'episode_count', 'work_count'] not found in axis"

## 4. 랜덤서치 CV

In [146]:
best_estimators = {}
best_scores = {}

# 1. Softmax Regression (Logistic Regression)
print("Tuning Logistic Regression (Softmax)...")
logistic_param_grid = {
    'C': [0.1, 1, 10],  # 정규화 강도
    'solver': ['lbfgs', 'newton-cg'],  # 최적화 알고리즘
    'multi_class': ['multinomial'],  # 소프트맥스 사용
    'max_iter': [100, 200]  # 최대 반복 횟수
}
logistic_model = LogisticRegression()
logistic_search = GridSearchCV(logistic_model, logistic_param_grid, cv=3, scoring='accuracy', verbose=1)
logistic_search.fit(X_train, y_train)
best_estimators['LogisticRegression'] = logistic_search.best_estimator_
best_scores['LogisticRegression'] = logistic_search.best_score_

# 2. Decision Tree
print("Tuning Decision Tree...")
tree_param_grid = {
    'criterion': ['gini', 'entropy'],  # 분할 기준
    'max_depth': [3, 5, None],  # 최대 깊이
    'min_samples_split': [2, 5, 10]  # 분할 최소 샘플 수
}
tree_model = DecisionTreeClassifier(random_state=21)
tree_search = GridSearchCV(tree_model, tree_param_grid, cv=3, scoring='accuracy', verbose=1)
tree_search.fit(X_train, y_train)
best_estimators['DecisionTree'] = tree_search.best_estimator_
best_scores['DecisionTree'] = tree_search.best_score_

# 3. Random Forest
print("Tuning Random Forest...")
forest_param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [3, 5, None],  # 최대 깊이
    'min_samples_split': [2, 5, 10],  # 분할 최소 샘플 수
    'criterion': ['gini', 'entropy']  # 분할 기준
}
forest_model = RandomForestClassifier(random_state=21)
forest_search = GridSearchCV(forest_model, forest_param_grid, cv=3, scoring='accuracy', verbose=1)
forest_search.fit(X_train, y_train)
best_estimators['RandomForest'] = forest_search.best_estimator_
best_scores['RandomForest'] = forest_search.best_score_

# 최적 모델과 결과 출력
print("\nBest Models and Scores:")
for model_name, model in best_estimators.items():
    print(f"{model_name}:")
    print(f"  Best Parameters: {model}")
    print(f"  Best CV Score: {best_scores[model_name]:.4f}")

# 최적 모델로 테스트 세트 평가
print("\nTest Set Performance:")
for model_name, model in best_estimators.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred, target_names=target_name))

Tuning Logistic Regression (Softmax)...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.or

Tuning Decision Tree...
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Tuning Random Forest...
Fitting 3 folds for each of 54 candidates, totalling 162 fits

Best Models and Scores:
LogisticRegression:
  Best Parameters: LogisticRegression(C=0.1, max_iter=200, multi_class='multinomial',
                   solver='newton-cg')
  Best CV Score: 0.6070
DecisionTree:
  Best Parameters: DecisionTreeClassifier(max_depth=3, random_state=21)
  Best CV Score: 0.6069
RandomForest:
  Best Parameters: RandomForestClassifier(min_samples_split=5, random_state=21)
  Best CV Score: 0.6122

Test Set Performance:
LogisticRegression Test Accuracy: 0.6345
              precision    recall  f1-score   support

           4       0.71      0.79      0.75        34
           2       0.53      0.60      0.57        53
           3       0.64      0.68      0.66        93
           5       0.65      0.55      0.59        44
           1       1.00      0.36      0.53        14

    accuracy     

### 4-1. boosting + 랜덤 서치 

In [148]:
dict_y = {'1':0 , '2':1, '3':2, '4':3, '5':4 }

y_test_2 = list(map(lambda x : dict_y[x], y_test))
y_train_2= list(map(lambda x : dict_y[x], y_train))

In [149]:
# XGBoost에서 1번 GPU 사용 강제 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print(f"CUDA_VISIBLE_DEVICES set to: {os.environ['CUDA_VISIBLE_DEVICES']}")

# XGBoost 2.1.0에서 1번 GPU 강제 사용
xgb_model = XGBClassifier(
    eval_metric="mlogloss",
    random_state=16,
    device="cuda:1",
    tree_method='gpu_hist'  # GPU 가속 전용 설정
)

# 랜덤 서치 파라미터 설정
xgb_params = {
    "n_estimators": [100, 200, 300, 400, 500],  # 트리 개수
    "learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],  # 학습률
    "max_depth": [3, 5, 7, 9, 11],  # 트리 깊이
    "subsample": [0.8, 0.9, 1.0],  # 샘플링 비율
    "colsample_bytree": [0.8, 0.9, 1.0],  # 피처 샘플링 비율
    "gamma": [0, 0.1, 0.2],  # 분할 시 최소 손실 감소 기준 (과적합 방지)
    "min_child_weight": [1, 3, 5, 7, 9],  # 리프 노드의 최소 가중치 (과적합 방지)
    "reg_alpha": [0, 0.01, 0.1, 0.5, 1],  # L1 정규화 (과적합 방지)
    "reg_lambda": [0, 0.1, 1, 5, 10]  # L2 정규화 (과적합 방지)
}

# RandomizedSearchCV 설정
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_params,
    n_iter=50,
    scoring="f1_macro",
    cv=3,
    verbose=2,
    n_jobs=-1,  # 병렬 실행
    random_state=16
)

# 모델 학습
random_search_xgb.fit(X_train, y_train_2)

# 최적의 모델 및 파라미터 출력
print(f"Best Parameters for XGBoost: {random_search_xgb.best_params_}")
print(f"Best Cross-Validation F1 Macro for XGBoost: {random_search_xgb.best_score_:.4f}")

# 테스트 셋 성능 평가
best_xgb_model = random_search_xgb.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test)

print("Classification Report for XGBoost:\n")
print(classification_report(y_test_2, y_pred_xgb))

CUDA_VISIBLE_DEVICES set to: 1
Fitting 3 folds for each of 50 candidates, totalling 150 fits


/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:15] WARNING: /workspace/src/context.cc:52: Only 1 GPUs are visible, setting device ordinal to 0
  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:15] WARNING: /workspace/src/context.cc:52: Only 1 GPUs are visible, setting device ordinal to 0
  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:15] WARNING: /works

[CV] END colsample_bytree=1.0, gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=5, n_estimators=200, reg_alpha=0, reg_lambda=0, subsample=0.8; total time=   2.3s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_depth=7, min_child_weight=5, n_estimators=300, reg_alpha=0.1, reg_lambda=5, subsample=0.9; total time=   0.0s


/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:18] WARNING: /workspace/src/context.cc:52: Only 1 GPUs are visible, setting device ordinal to 0
  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:18] WARNING: /workspace/src/context.cc:52: Only 1 GPUs are visible, setting device ordinal to 0
  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:18:18] WARNING: /works

[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=500, reg_alpha=0.1, reg_lambda=1, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=500, reg_alpha=0.1, reg_lambda=1, subsample=0.8; total time=   0.0s


...

[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_depth=9, min_child_weight=3, n_estimators=400, reg_alpha=0, reg_lambda=0, subsample=1.0; total time= 4.3min


/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:22:40] WARNING: /workspace/src/context.cc:52: Only 1 GPUs are visible, setting device ordinal to 0
  warnings.warn(smsg, UserWarning)
/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:22:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Best Parameters for XGBoost: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 400, 'min_child_weight': 7, 'max_depth': 7, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.8}
Best Cross-Validation F1 Macro for XGBoost: 0.5709


/home/wanted-1/miniconda3/envs/project2_team1/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [06:25:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Classification Report for XGBoost:

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        34
           1       0.64      0.64      0.64        53
           2       0.67      0.73      0.70        93
           3       0.58      0.59      0.58        44
           4       0.80      0.29      0.42        14

    accuracy                           0.66       238
   macro avg       0.69      0.60      0.62       238
weighted avg       0.67      0.66      0.66       238



In [150]:
# GPU 디바이스 확인
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 1번 GPU 사용
print(f"CUDA_VISIBLE_DEVICES set to: {os.environ['CUDA_VISIBLE_DEVICES']}")

# 랜덤 서치 파라미터 설정
lgbm_params = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.2],
    "num_leaves": [31, 50, 100],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
}

# LGBM 모델 정의
lgbm_model = LGBMClassifier(random_state=17, eval_metrics = 'TotalF1')

# RandomizedSearchCV 설정
random_search_lgbm = RandomizedSearchCV(
    estimator=lgbm_model,
    param_distributions=lgbm_params,
    n_iter=3,
    scoring="f1_macro",
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=17
)

# 모델 학습
random_search_lgbm.fit(X_train, y_train_2)

# 최적의 모델 및 파라미터 출력
print(f"Best Parameters for LGBM: {random_search_lgbm.best_params_}")
print(f"Best Cross-Validation F1 Macro for LGBM: {random_search_lgbm.best_score_:.4f}")

# 테스트 셋 성능 평가
best_lgbm_model = random_search_lgbm.best_estimator_
y_pred_lgbm = best_lgbm_model.predict(X_test)
print("Classification Report for LGBM:\n")
print(classification_report(y_test_2, y_pred_lgbm))

CUDA_VISIBLE_DEVICES set to: 1
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[LightGBM] [Warning] Unknown parameter: eval_metrics
[LightGBM] [Warning] Unknown parameter: eval_metrics
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12015
[LightGBM] [Info] Number of data points in the train set: 632, number of used features: 67
[LightGBM] [Info] Start training from score -2.006238
[LightGBM] [Info] Start training from score -1.451677
[LightGBM] [Info] Start training from score -0.951721
[LightGBM] [Info] Start training from score -1.661398
[LightGBM] [Info] Start training from score -2.893541
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 